# Gateway IAM role Inbound를 사용하여 AWS Lambda를 MCP화하기
## Bedrock AgentCore Gateway를 사용하여 AWS Lambda 함수를 안전한 MCP Tool로 변환

## 개요
Bedrock AgentCore Gateway는 고객이 인프라나 호스팅을 관리할 필요 없이 기존 AWS Lambda 함수를 완전 관리형 MCP 서버로 전환할 수 있는 방법을 제공합니다. Gateway는 이러한 모든 Tool에 대해 통일된 Model Context Protocol (MCP) 인터페이스를 제공합니다. Gateway는 들어오는 요청과 대상 리소스로의 아웃바운드 연결 모두에 대한 안전한 액세스 제어를 보장하기 위해 이중 인증 모델을 사용합니다. 프레임워크는 두 가지 주요 구성 요소로 이루어져 있습니다: gateway 대상에 액세스하려는 사용자를 검증하고 권한을 부여하는 Inbound Auth와, 인증된 사용자를 대신하여 gateway가 백엔드 리소스에 안전하게 연결할 수 있도록 하는 Outbound Auth입니다. Gateway는 아웃바운드 권한 부여를 위해 IAM role을 사용하여 AWS Lambda 함수 호출을 승인합니다.

이 예제에서는 IAM role을 사용하여 인바운드 및 아웃바운드 권한 부여를 모두 시연합니다.

![How does it work](images/lambda-gw-iam-inbound.png)

### 튜토리얼 세부 정보


| Information          | Details                                                   |
|:---------------------|:----------------------------------------------------------|
| Tutorial type        | 인터랙티브                                               |
| AgentCore components | AgentCore Gateway                                         |
| Agentic Framework    | Strands Agents                                            |
| Gateway Target type  | AWS Lambda                                                |
| Inbound Auth         | AAWS IAM                                                  |
| Outbound Auth        | AWS IAM                                                   |
| LLM model            | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| Tutorial components  | AgentCore Gateway 생성 및 AgentCore Gateway 호출 |
| Tutorial vertical    | Cross-vertical                                            |
| Example complexity   | Easy                                                      |
| SDK used             | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 Lambda용 AmazonCore Gateway 대상을 생성합니다

### 튜토리얼 아키텍처
이 튜토리얼에서는 AWS lambda 함수에 정의된 작업을 MCP Tool로 변환하고 Bedrock AgentCore Gateway에서 호스팅합니다. AWS Sigv4 헤더의 AWS IAM 자격 증명을 사용하여 ingress auth를 시연합니다.
시연 목적으로 Amazon Bedrock Model을 사용하는 Strands Agent를 사용합니다.
예제에서는 get_order와 update_order라는 두 가지 Tool을 가진 매우 간단한 agent를 사용합니다.

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook (Python kernel)
* uv
* AWS credentials
* AWS console을 통한 Nova Pro 액세스
* Amazon Bedrock AgentCore SDK
* Strand Agents

## 들어오는 AgentCore Gateway 요청에 대한 인증 구성
AgentCore Gateway는 인바운드 및 아웃바운드 인증을 통해 안전한 연결을 제공합니다. 인바운드 인증의 경우, AgentCore Gateway는 이제 gateway를 호출하기 위해 OAuth 외에도 AWS IAM 자격 증명/ID를 지원합니다. Tool이 외부 리소스에 액세스해야 하는 경우, AgentCore Gateway는 API Key, IAM 또는 OAuth Token을 통한 아웃바운드 인증을 사용하여 외부 리소스에 대한 액세스를 허용하거나 거부할 수 있습니다.

인바운드 권한 부여 흐름 중에 agent 또는 MCP 클라이언트는 인증에 사용되고 AgentCore Gateway에 액세스하기 위한 IAM 권한에 대해 권한을 부여받는 AWS Signature V4 서명된 요청을 사용합니다. AgentCore Gateway는 AWS IAM 자격 증명/ID를 검증하고 인바운드 권한 부여를 수행합니다.

AgentCore Gateway에서 실행되는 Tool이 외부 리소스에 액세스해야 하는 경우, IAM role은 Gateway 대상에 대한 다운스트림 리소스의 자격 증명을 검색합니다. AgentCore Gateway는 다운스트림 API에 액세스하기 위해 권한 부여 자격 증명을 호출자에게 전달합니다.

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Set AWS credentials if not using Amazon SageMaker notebook
import os

# SageMaker 외부 환경에서 실행 시 AWS 자격 증명 설정 필요
#os.environ['AWS_ACCESS_KEY_ID']=''
#os.environ['AWS_SECRET_ACCESS_KEY']=''
os.environ['AWS_DEFAULT_REGION']='us-west-2' # set the AWS region




In [ ]:
import os
import sys

# Get the directory of the current script
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    # Jupyter 환경에서는 __file__이 정의되지 않으므로 현재 작업 디렉토리 사용
    current_dir = os.getcwd()  # Fallback if __file__ is not defined (e.g., Jupyter)

# Navigate to the directory containing utils.py (one level up)
# utils.py가 있는 상위 디렉토리로 이동
utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add to sys.path
# Python 모듈 검색 경로에 추가
sys.path.insert(0, utils_dir)



In [ ]:
# Now you can import utils
import utils
#### Create a sample AWS Lambda function that you want to convert into MCP tools
# MCP tool로 변환할 샘플 Lambda 함수 생성
lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")
if lambda_resp is not None:
    if lambda_resp['exit_code'] == 0:
        print("Lambda function created with ARN: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda function creation failed with message: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Create an IAM role for the Gateway to assume
# Gateway가 assume할 IAM role 생성
import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore gateway role ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# 인바운드 권한 부여를 위한 Amazon IAM Authorizer로 Gateway 생성

In [ ]:
import time
import boto3
# CreateGateway with Amazon IAM. 
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])

# AWS IAM 인증을 사용하는 Gateway 생성
create_response = gateway_client.create_gateway(name='TestGWforLambdaIAM',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # The IAM Role must have permissions to create/list/get/delete Gateway 
    protocolType='MCP',
    authorizerType='AWS_IAM',  # IAM 기반 인증 사용
    description='AgentCore Gateway with AWS Lambda target type using Amazon IAM for ingress auth'
)
print(create_response)
# Retrieve the GatewayID used for GatewayTarget creation
# GatewayTarget 생성에 사용할 GatewayID 추출
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)
time.sleep(10)

# AWS Lambda 대상을 생성하고 MCP Tool로 변환

In [ ]:
# Replace the AWS Lambda function ARN below
# Lambda 함수의 설정 및 MCP tool schema 정의
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # Replace this with your AWS Lambda function ARN
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "tool to get the order",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "tool to update the orderId",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

# Gateway IAM role을 사용한 자격 증명 설정
credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
# Gateway target 생성 (Lambda 함수를 MCP tool로 등록)
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Gateway를 호출하기 위한 AWS IAM Role 생성

아래 함수는 AWS Bedrock AgentCore가 지정된 gateway를 호출할 수 있도록 하는 IAM role을 생성하거나 업데이트합니다. 주어진 gateway ID에 대해 bedrock-agentcore:InvokeGateway 권한을 부여하는 인라인 정책을 구축하고 연결합니다. 또한 Bedrock AgentCore 서비스와 호출하는 IAM 엔티티(current_arn) 모두가 role을 assume할 수 있도록 허용하는 신뢰 정책을 구성합니다.

In [ ]:
#### Create an IAM role to Invoke Gateway
# 현재 실행 중인 IAM role ARN 가져오기
current_role_arn = utils.get_current_role_arn()
print("Current role ARN: ", current_role_arn)

# Gateway 호출 권한을 가진 IAM role 생성
agentcore_gateway_iam_invoke_role = utils.create_gateway_invoke_tool_role("gateway-invoke-role",gatewayID , current_role_arn)
print("Role to invoke Agentcore gateway ARN: ", agentcore_gateway_iam_invoke_role['Role']['Arn'])

# Bedrock AgentCore Gateway를 사용하여 AWS Lambda의 MCP Tool을 호출하는 Strands agent

#### MCP Client SDK에서 AWS IAM 인증 지원

AgentCore Gateway에 대한 인바운드 요청에 AWS IAM 인증이 이제 지원되지만, 현재 오픈 소스 MCP Client SDK는 스트리밍 가능한 HTTP 연결에 대한 SigV4 인증 지원이 제한적이라는 점에 유의해야 합니다. 그러나 AWS는 "Run Model Context Protocol (MCP) servers with AWS Lambda" 프로젝트를 통해 솔루션을 제공했으며, 여기에는 스트리밍 HTTP 연결을 위한 SigV4 인증의 중요한 확장이 포함되어 있습니다.

[AWS Labs GitHub 리포지토리](https://github.com/awslabs/run-model-context-protocol-servers-with-aws-lambda/tree/main)에서 사용할 수 있는 이 구현은 스트리밍 연결에 대한 인증 격차를 해소하며 Strands 또는 LangChain과 같은 인기 있는 agentic 프레임워크와 원활하게 통합될 수 있습니다. StreamableHTTPTransportWithSigV4 클래스는 표준 MCP transport 레이어를 확장하여 스트리밍 기능을 유지하면서 AWS SigV4 서명을 처리하므로 AgentCore Gateway의 새로운 IAM 인증 기능과 호환됩니다.

In [ ]:
!pip3 install --upgrade strands-agents strands-agents-tools
from strands.models import BedrockModel

## The IAM credentials configured in ~/.aws/credentials should have access to Bedrock model
# Bedrock model 초기화 (Nova Pro 사용)
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging
from strands import Agent
import logging
from strands.tools.mcp.mcp_client import MCPClient
from mcp.client.streamable_http import streamablehttp_client 
from botocore.credentials import Credentials
from streamable_http_sigv4 import (
    streamablehttp_client_with_sigv4,
)

SERVICE="bedrock-agentcore"

# Configure the root strands logger. Change it to DEBUG if you are debugging the issue.
logging.getLogger("strands").setLevel(logging.INFO)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Bearer token 기반 HTTP transport 생성 (OAuth 사용 시)
def create_streamable_http_transport(mcp_url: str, access_token: str):
       return streamablehttp_client(mcp_url, headers={"Authorization": f"Bearer {access_token}"})

# AWS SigV4 인증을 사용하는 HTTP transport 생성 (IAM 인증 사용 시)
def create_streamable_http_transport_sigv4(mcp_url: str, key: str, secret: str, sessionToken: str, serviceName: str, awsRegion: str):
        iamcredentials = Credentials(
            access_key=key,
            secret_key=secret,
            token = sessionToken
        )
        return streamablehttp_client_with_sigv4(
            url=mcp_url,
            credentials=iamcredentials,
            service=serviceName,
            region=awsRegion,
        )

# MCP client에서 모든 tool 목록 가져오기 (pagination 처리)
def get_full_tools_list(client):
    more_tools = True
    tools = []
    pagination_token = None
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)
        tools.extend(tmp_tools)
        if tmp_tools.pagination_token is None:
            more_tools = False
        else:
            more_tools = True 
            pagination_token = tmp_tools.pagination_token
    return tools

# MCP tool 동기 호출 및 결과 추출
def call_tool_sync(client, tool_id,tool_name, parameters=None):
    # Call the tool (no pagination argument supported)
    response = client.call_tool_sync(
        tool_use_id=tool_id,
        name=tool_name,
        arguments=parameters
    )

    # Extract output content
    # 응답 형식에 따라 결과 추출
    if hasattr(response, "results") and response.results:
        return response.results
    elif hasattr(response, "output") and response.output:
        return response.output
    elif hasattr(response, "content"):
        return response.content
    else:
        return response  # fallback
         
# Agent 실행: MCP client 초기화 및 tool 호출
def run_agent(mcp_url: str,key: str, secret: str, sessionToken: str,serviceName: str, awsRegion: str):
    # SigV4 인증을 사용하는 MCP client 생성
    mcp_client = MCPClient(lambda: create_streamable_http_transport_sigv4(mcp_url,key,secret,sessionToken,serviceName, awsRegion))

    with mcp_client:
        # Gateway에서 사용 가능한 tool 목록 조회
        tools = get_full_tools_list(mcp_client)
        print(f"Found the following tools: {[tool.tool_name for tool in tools]}")
        print(f"Tool name: {tools[0].tool_name}")
        
        # Strands Agent에 tool 등록 및 실행
        agent = Agent(model=yourmodel,tools=tools) ## you can replace with any model you like
        print(f"Tools loaded in the agent are {agent.tool_names}")
        agent("Check the order status for order id 123 and show me the exact response from the tool")
        #call mcp with tool
        # MCP tool 직접 호출 예제
        tool=tools[0].tool_name
        tool_id="get-order-id-123-call-1"
        result = call_tool_sync(
            mcp_client,
            tool_id,
            tool_name=tool,
            parameters={"orderId": "123"}
        )

        print(f"Tool Call result: {result['content'][0]['text']}")
        

IAM Gateway Invoke role을 assume하고 Agent 실행

In [ ]:
sts_client = boto3.client("sts")
# Gateway 호출 권한을 가진 role로 임시 자격 증명 획득
response = sts_client.assume_role(
        RoleArn=agentcore_gateway_iam_invoke_role['Role']['Arn'],
        RoleSessionName="invoke_mcp_session",
        DurationSeconds=3600  # 1 hour, can be up to 12h for some roles
)

# 임시 자격 증명 추출
creds = response["Credentials"]

access=creds["AccessKeyId"]
secret=creds["SecretAccessKey"]
token=creds["SessionToken"]

# Run Agent with the credentials from this new gateway-invoke-role
# 획득한 임시 자격 증명으로 Agent 실행
time.sleep(10) 
run_agent(gatewayURL,access,secret,token, SERVICE, os.environ['AWS_DEFAULT_REGION'])

**문제: 아래 셀을 실행할 때 다음 오류가 발생하면 pydantic과 pydantic-core 버전 간의 비호환성을 나타냅니다.**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**해결 방법은?**

호환되는 pydantic==2.7.2와 pydantic-core 2.27.2가 모두 있는지 확인해야 합니다. 완료되면 커널을 다시 시작하세요.

# 정리

IAM role, IAM Policies, Credentials provider 및 AWS Lambda 함수와 같은 추가 리소스도 생성되며, 정리의 일부로 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## gateway 삭제 (선택 사항)

In [ ]:
import utils
# Gateway 및 관련 리소스 삭제
utils.delete_gateway(gateway_client,gatewayID)